## Train step 1: Bootstrap from pretrained model

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2
%matplotlib notebook

import pickle, os, warnings, sys, random, logging, librosa, json, nemo
warnings.filterwarnings("ignore")
logger = logging.getLogger()
logger.setLevel(logging.INFO)

from Cfg import Cfg
from reshuffle_samples import reshuffle_samples
import nemo.collections.asr as nemo_asr
from ruamel.yaml import YAML

from omegaconf import DictConfig
import os, datetime
from load_pretrained_amharic_model import load_pretrained_amharic_model

from reshuffle_samples import reshuffle_samples
import pytorch_lightning as pl

[NeMo W 2020-10-15 14:45:43 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToCharDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-15 14:45:43 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToBPEDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-15 14:45:43 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioLabelDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-15 14:45:43 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text._TarredAudioToTextDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-15 14:45:43 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.TarredAudioToCharDataset'> is experimental,

In [2]:
C, model = load_pretrained_amharic_model()

[NeMo W 2020-10-15 14:45:45 modelPT:102] Please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: NIST/openasr20_amharic/build/train_manifest.json
    sample_rate: 16000
    labels:
    - ' '
    - ሀ
    - ሁ
    - ሂ
    - ሃ
    - ሄ
    - ህ
    - ሆ
    - ለ
    - ሉ
    - ሊ
    - ላ
    - ሌ
    - ል
    - ሎ
    - ሏ
    - ሐ
    - ሑ
    - ሒ
    - ሔ
    - ሕ
    - መ
    - ሙ
    - ሚ
    - ማ
    - ሜ
    - ም
    - ሞ
    - ሟ
    - ሠ
    - ሡ
    - ሣ
    - ሥ
    - ሦ
    - ረ
    - ሩ
    - ሪ
    - ራ
    - ሬ
    - ር
    - ሮ
    - ሯ
    - ሰ
    - ሱ
    - ሲ
    - ሳ
    - ሴ
    - ስ
    - ሶ
    - ሷ
    - ሸ
    - ሹ
    - ሺ
    - ሻ
    - ሼ
    - ሽ
    - ሾ
    - ሿ
    - ቀ
    - ቁ
    - ቂ
    - ቃ
    - ቄ
    - ቅ
    - ቆ
    - ቋ
    - በ
    - ቡ
    - ቢ
    - ባ
    - ቤ
    - ብ
    - ቦ
    - ቧ
    - ቨ
    - ቪ
    - ቫ
    - ቬ
    - ቭ
    - ተ
    - ቱ
    - ቲ
    - ታ
    - ቴ
    - ት
    - ቶ
    - ቷ
    - ቸ
  

[NeMo I 2020-10-15 14:45:45 features:241] PADDING: 16
[NeMo I 2020-10-15 14:45:45 features:258] STFT using torch
loaded save/nemo_amharic/amharic_20201015_005720_353924_epoch=199.ckpt


In [ ]:
config_path = 'amharic_16000.yaml'
yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
train_manifest=f'{C.build_dir}/train_manifest.json'
test_manifest=f'{C.build_dir}/test_manifest.json'
params['model']['train_ds']['manifest_filepath'] = train_manifest
params['model']['validation_ds']['manifest_filepath'] = test_manifest

model.setup_training_data(train_data_config=params['model']['train_ds'])
model.setup_validation_data(val_data_config=params['model']['validation_ds'])

In [ ]:
model_save_dir='save/nemo_amharic'

class ModelCheckpointAtEpochEnd(pl.callbacks.ModelCheckpoint):
    def on_epoch_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        metrics['epoch'] = trainer.current_epoch
        trainer.checkpoint_callback.on_validation_end(trainer, pl_module)

pid=os.getpid()
dt=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

checkpoint_callback = ModelCheckpointAtEpochEnd(
    filepath=model_save_dir+'/amharic_'+f'{dt}_{pid}'+'_{epoch:02d}',
    verbose=True,
    save_top_k=-1,
    save_weights_only=False,
    period=1)

In [5]:
trainer = pl.Trainer(gpus=[0], max_epochs=200, amp_level='O1', precision=16, checkpoint_callback=checkpoint_callback)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


In [9]:
params['model']['optim']['lr']=0.001

In [10]:
model.setup_optimization(optim_config=DictConfig(params['model']['optim']))

[NeMo I 2020-10-15 16:16:44 modelPT:572] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.001
        weight_decay: 0.001
    )


[NeMo W 2020-10-15 16:16:44 lr_scheduler:526] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


(Novograd (
 Parameter Group 0
     amsgrad: False
     betas: [0.8, 0.5]
     eps: 1e-08
     grad_averaging: False
     lr: 0.001
     weight_decay: 0.001
 ),
 None)

In [11]:
reshuffle_samples(C)

In [ ]:
trainer.fit(model)

[NeMo I 2020-10-15 16:18:02 modelPT:572] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.001
        weight_decay: 0.001
    )


[NeMo W 2020-10-15 16:18:02 lr_scheduler:526] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18 M  
2 | decoder           | ConvASRDecoder                    | 241 K 
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     


[NeMo I 2020-10-15 16:18:02 wer:148] 
    
[NeMo I 2020-10-15 16:18:02 wer:149] reference:ማታ
[NeMo I 2020-10-15 16:18:02 wer:150] decoded  :መቶ


[NeMo I 2020-10-15 16:18:02 wer:148] 
    
[NeMo I 2020-10-15 16:18:02 wer:149] reference:አይ አልደወልኩም ቆይቻለሁ ከደወልኩኝ
[NeMo I 2020-10-15 16:18:02 wer:150] decoded  :አይ አልደወልኩምቆይቻለሁ ከደወልኩኝ
[NeMo I 2020-10-15 16:18:03 wer:148] 
    
[NeMo I 2020-10-15 16:18:03 wer:149] reference:እነ እታለም እንደት ናቸው ደህና ናቸው
[NeMo I 2020-10-15 16:18:03 wer:150] decoded  :እነ እታለም እንደት ናቸው ደህና ናቸው
[NeMo I 2020-10-15 16:18:03 wer:148] 
    
[NeMo I 2020-10-15 16:18:03 wer:149] reference:ለምንድን
[NeMo I 2020-10-15 16:18:03 wer:150] decoded  :ማምንድን
[NeMo I 2020-10-15 16:18:03 wer:148] 
    
[NeMo I 2020-10-15 16:18:03 wer:149] reference:ሥራ እንደት አድርጎሻል
[NeMo I 2020-10-15 16:18:03 wer:150] decoded  :ሥራ እንደት አድርጎሻል
[NeMo I 2020-10-15 16:18:04 wer:148] 
    
[NeMo I 2020-10-15 16:18:04 wer:149] reference:በእውነት የት ሆንኩ አለችሽ
[NeMo I 2020-10-15 16:18:04 wer:150] decoded  :በእውነት የሱ እንኩ አለችሽ
[NeMo I 2020-10-15 16:18:04 wer:148] 
    
[NeMo I 2020-10-15 16:18:04 wer:149] reference:ለእዛ ይሆናል ለነገሩ ምን እሷ ብቻ አንቺም ወፍረሻል
[NeMo I 2020-10-

[NeMo I 2020-10-15 16:18:23 wer:148] 
    
[NeMo I 2020-10-15 16:18:23 wer:149] reference:እ
[NeMo I 2020-10-15 16:18:23 wer:150] decoded  :እ
[NeMo I 2020-10-15 16:18:23 wer:148] 
    
[NeMo I 2020-10-15 16:18:23 wer:149] reference:ኦኬ ምን ዐይነት እ
[NeMo I 2020-10-15 16:18:23 wer:150] decoded  :ኦኬ ምን ዐይነት እ
[NeMo I 2020-10-15 16:18:24 wer:148] 
    
[NeMo I 2020-10-15 16:18:24 wer:149] reference:ሄሎ ሄሎ አይሰማሽም የምትሄደው
[NeMo I 2020-10-15 16:18:24 wer:150] decoded  :ሄሎ ሄሎ አይሰማሽም የምትሄደው
[NeMo I 2020-10-15 16:18:24 wer:148] 
    
[NeMo I 2020-10-15 16:18:24 wer:149] reference:እንደ እኔ ጓደኛው አለኝ አይደል እንደ
[NeMo I 2020-10-15 16:18:24 wer:150] decoded  :እንደ እኔ ጓደኛው አለኝ አይደል እንደ
[NeMo I 2020-10-15 16:18:25 wer:148] 
    
[NeMo I 2020-10-15 16:18:25 wer:149] reference:ኧረ አረረረ
[NeMo I 2020-10-15 16:18:25 wer:150] decoded  :ኧረ  አረረ
[NeMo I 2020-10-15 16:18:25 wer:148] 
    
[NeMo I 2020-10-15 16:18:25 wer:149] reference:አቤት አቤት እየሰማሁህ ነው
[NeMo I 2020-10-15 16:18:25 wer:150] decoded  :አቤት አቤት እየሰማሁህ ነው
[NeMo 

...

[NeMo I 2020-10-15 16:34:03 wer:148] 
    
[NeMo I 2020-10-15 16:34:03 wer:149] reference:እ እ ይጀምራሉ
[NeMo I 2020-10-15 16:34:03 wer:150] decoded  :እ እ ላ ጀብለች
[NeMo I 2020-10-15 16:34:04 wer:148] 
    
[NeMo I 2020-10-15 16:34:04 wer:149] reference:ሰላም ነው አምብዬ
[NeMo I 2020-10-15 16:34:04 wer:150] decoded  :ሰላም ነው አምብዬ
[NeMo I 2020-10-15 16:34:04 wer:148] 
    
[NeMo I 2020-10-15 16:34:04 wer:149] reference:ስለዚህ ያው ሰውነት ምንም አያስፈልግህም ቁመትም አያስፈልግህም ያለህ ይበቃሀል አይደል
[NeMo I 2020-10-15 16:34:04 wer:150] decoded  :ስለዚህያ ያው ሰውነት ምንም አያስፈልግህም ቁመትም አያስፈልግህም ያለህ ይበቃሀል አይደል
[NeMo I 2020-10-15 16:34:05 wer:148] 
    
[NeMo I 2020-10-15 16:34:05 wer:149] reference:የተቸገረ የተቸገረ መርዳት ባህላችን ነው
[NeMo I 2020-10-15 16:34:05 wer:150] decoded  :የተቸገረ የተቸገረ መርዳት ባህላችን ነው
[NeMo I 2020-10-15 16:34:05 wer:148] 
    
[NeMo I 2020-10-15 16:34:05 wer:149] reference:አላውቅብህም ግን እንደዚያ እያልከኝ አይደል እንደ
[NeMo I 2020-10-15 16:34:05 wer:150] decoded  :አዎውቅብህ ግን እንደዚያ እያልከኝ አይደል እንደ
[NeMo I 2020-10-15 16:34:06 wer:148] 
    
[

[NeMo I 2020-10-15 16:34:24 wer:149] reference:እ አስበላሁሽ
[NeMo I 2020-10-15 16:34:24 wer:150] decoded  :እ አስበላሁሽ
[NeMo I 2020-10-15 16:34:24 wer:148] 
    
[NeMo I 2020-10-15 16:34:24 wer:149] reference:እንደዚያ ሲሉ ነበረ መሰለኝ እንደ ለእዛ ብዬሽ እኮ ነው እ
[NeMo I 2020-10-15 16:34:24 wer:150] decoded  :እንደዚያ ሲሉ ነበረ መሰለኝ እንደ ለእዛ ብዬሽ እኮ ነው እ
[NeMo I 2020-10-15 16:34:25 wer:148] 
    
[NeMo I 2020-10-15 16:34:25 wer:149] reference:አዎ በነገርሽ ላይ
[NeMo I 2020-10-15 16:34:25 wer:150] decoded  :አዎ በነገርሽ ላይ
[NeMo I 2020-10-15 16:34:25 wer:148] 
    
[NeMo I 2020-10-15 16:34:25 wer:149] reference:አዎ
[NeMo I 2020-10-15 16:34:25 wer:150] decoded  :አዎ
[NeMo I 2020-10-15 16:34:25 wer:148] 
    
[NeMo I 2020-10-15 16:34:25 wer:149] reference:ኧረ እባክህ
[NeMo I 2020-10-15 16:34:25 wer:150] decoded  :ኧረ እባክህ
[NeMo I 2020-10-15 16:34:26 wer:148] 
    
[NeMo I 2020-10-15 16:34:26 wer:149] reference:በቃ እ እንደት ነሽ ሰላም ነው
[NeMo I 2020-10-15 16:34:26 wer:150] decoded  :በቃ እ እንደት ነሽ ሰላም ነው
[NeMo I 2020-10-15 16:34:26 wer:148] 
   

[NeMo I 2020-10-15 16:34:45 wer:149] reference:አቤት
[NeMo I 2020-10-15 16:34:45 wer:150] decoded  :አቤት
[NeMo I 2020-10-15 16:34:45 wer:148] 
    
[NeMo I 2020-10-15 16:34:45 wer:149] reference:እይ እንደ እኔ እራ እኔ እራሱ ነገሩ አሳየው ገና አክቱ ገርሞኛል ልክ ነሽ
[NeMo I 2020-10-15 16:34:45 wer:150] decoded  :እንደ እኔ እራ እኔ እራሱ ነገሩ አሳየው ገና አክቱ ገርሞኛል ልክ ነሽ
[NeMo I 2020-10-15 16:34:46 wer:148] 
    
[NeMo I 2020-10-15 16:34:46 wer:149] reference:ምን ሆነብኝ
[NeMo I 2020-10-15 16:34:46 wer:150] decoded  :ም ነብኝ
[NeMo I 2020-10-15 16:34:46 wer:148] 
    
[NeMo I 2020-10-15 16:34:46 wer:149] reference:ነበረ አልተገናኙማ ያቺኛዋ ሥራ ናት እሑድ እሑድ አትኖር እሑድ እሑድ ነው ብዙዬ ደግሞ እረፍቷ
[NeMo I 2020-10-15 16:34:46 wer:150] decoded  :ነበረ አልተገናኙማ ያቺኛዋ ሥራ ናት እሑድ እድ አትኖ ሑድሑድ  ነው ብዙዬ ደግሞ እረፍቷ
[NeMo I 2020-10-15 16:34:47 wer:148] 
    
[NeMo I 2020-10-15 16:34:47 wer:149] reference:እ
[NeMo I 2020-10-15 16:34:47 wer:150] decoded  :እ
[NeMo I 2020-10-15 16:34:47 wer:148] 
    
[NeMo I 2020-10-15 16:34:47 wer:149] reference:እህትሽ
[NeMo I 2020-10-15 16:34:47 

[NeMo I 2020-10-15 16:35:05 wer:150] decoded  :ሥራ ጥሩ ነው ያው እንግዲህ በቀን አስራ ሁለት ሰዐት እንሰራለን እናንተ ጋር እንደት ነው ሥራ
[NeMo I 2020-10-15 16:35:06 wer:148] 
    
[NeMo I 2020-10-15 16:35:06 wer:149] reference:እና አይ እርከን አለው ምድብ አንድ ምድብ ሁለት ምናምን አለ አይደል
[NeMo I 2020-10-15 16:35:06 wer:150] decoded  :እና አይ እርከን አለው ምድብ አንድ ምድብ ሁለት ምናምን አለ አይደል
[NeMo I 2020-10-15 16:35:06 wer:148] 
    
[NeMo I 2020-10-15 16:35:06 wer:149] reference:ስልክ ሞክሬ አልሰራኝም እያልከኝ ነበር
[NeMo I 2020-10-15 16:35:06 wer:150] decoded  :ስልክ ሞክሬ አልሰራኝም እያልከኝ ነበር
[NeMo I 2020-10-15 16:35:07 wer:148] 
    
[NeMo I 2020-10-15 16:35:07 wer:149] reference:ሄሎ
[NeMo I 2020-10-15 16:35:07 wer:150] decoded  :ሄሎ
[NeMo I 2020-10-15 16:35:07 wer:148] 
    
[NeMo I 2020-10-15 16:35:07 wer:149] reference:እ
[NeMo I 2020-10-15 16:35:07 wer:150] decoded  :እ
[NeMo I 2020-10-15 16:35:08 wer:148] 
    
[NeMo I 2020-10-15 16:35:08 wer:149] reference:ሄሎ
[NeMo I 2020-10-15 16:35:08 wer:150] decoded  :እ ት
[NeMo I 2020-10-15 16:35:08 wer:148] 
    
[NeMo I 20

[NeMo I 2020-10-15 16:35:27 wer:148] 
    
[NeMo I 2020-10-15 16:35:27 wer:149] reference:አዎ ሶስተኛው ላይ ሂጅ ወይም ደግሞ
[NeMo I 2020-10-15 16:35:27 wer:150] decoded  :አዎ ሶስተኛው ላይ ሂጅ ወይም ደግሞ
[NeMo I 2020-10-15 16:35:27 wer:148] 
    
[NeMo I 2020-10-15 16:35:27 wer:149] reference:ኦው ኦው
[NeMo I 2020-10-15 16:35:27 wer:150] decoded  :ኦው ኦው
[NeMo I 2020-10-15 16:35:28 wer:148] 
    
[NeMo I 2020-10-15 16:35:28 wer:149] reference:አዎ
[NeMo I 2020-10-15 16:35:28 wer:150] decoded  :አዎ
[NeMo I 2020-10-15 16:35:28 wer:148] 
    
[NeMo I 2020-10-15 16:35:28 wer:149] reference:አይዞሽ አይዞሽ እንግዲህ ምን ታደርጊዋለሽ እ
[NeMo I 2020-10-15 16:35:28 wer:150] decoded  :አይዞሽ አይዞሽ እንግዲህ ምን ታደርጊዋለሽ እ
[NeMo I 2020-10-15 16:35:29 wer:148] 
    
[NeMo I 2020-10-15 16:35:29 wer:149] reference:እዘ እዚህ ና በ እዚህ ና
[NeMo I 2020-10-15 16:35:29 wer:150] decoded  :እግ ነ እዚህ ና በ እዚህ ና
[NeMo I 2020-10-15 16:35:29 wer:148] 
    
[NeMo I 2020-10-15 16:35:29 wer:149] reference:እሺ
[NeMo I 2020-10-15 16:35:29 wer:150] decoded  :እሺ
[NeMo I 2020-1